# SQL codes Used on BigQuery

## 1. vehicles

n(HH) by Veh by MPO

In [ ]:
SELECT
  polygons_table.mpo AS mpo,
  points_table.vehicles AS vehicles,
  COUNT(DISTINCT household_id) AS num_households
FROM
  `project-using-replica.tdm_calib23.mpo_geo` AS polygons_table
JOIN
  `replica-customer.northeast.northeast_2023_Q4_population` AS points_table
ON
  ST_CONTAINS(polygons_table.geo_gg, ST_GEOGPOINT(points_table.lng, points_table.lat))
WHERE 
  points_table.lat IS NOT NULL
GROUP BY
  polygons_table.mpo, points_table.vehicles;

> output: '.\data\mpo_veh_Nhh_0813.csv'

## 2. Check

### 2.1 group quarters (GQ) [.]

vehicles = 'GQ'

or

household_size = '1_person_group_quarters'

### 2.2 household_size vs person_id [?]

In [ ]:
SELECT
    household_id,
    household_size,
    count(person_id) AS person_count
FROM

    `replica-customer.northeast.northeast_2023_Q4_population`

WHERE 
    -- points_table.lat IS NOT NULL 
    -- AND 
    household_size = '6_person'
GROUP BY
    household_id, household_size
ORDER BY person_count ASC
limit 100;

> example output: household_id, household_size = '6_persons', person_count = 1

7 plus

20

## 3. Vehicle Availability (VA)

n(HH) by VA by MPO


## 3.1 no GQ

## 3.2 vehicle_count

## 3.3 adult_count

allows 

    1. filters on adult_count
    2. changes on vehicle_count (e.g., 3 into 3.5)

In [ ]:
CREATE TABLE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4` AS 
    SELECT
        points_table.household_id,
        polygons_table.mpo,
        MAX(CASE points_table.vehicles
            WHEN 'zero' THEN 0
            WHEN '1' THEN 1
            WHEN '2' THEN 2
            WHEN '3_plus' THEN 3 -- Assuming '3_plus' means at least 3 vehicles
            ELSE 0
            END) AS vehicle_count,
        SUM(CASE WHEN age >= 18 THEN 1 ELSE 0 END) AS adult_count
    FROM
        `project-using-replica.tdm_calib23.mpo_geo` AS polygons_table
    JOIN
        `replica-customer.northeast.northeast_2023_Q4_population` AS points_table
    ON
        ST_CONTAINS(polygons_table.geo_gg, ST_GEOGPOINT(points_table.lng, points_table.lat))
    WHERE 
        points_table.vehicles <> 'GQ' AND points_table.lat IS NOT NULL 
    GROUP BY
        polygons_table.mpo, points_table.household_id;
    -- ORDER BY adult_count DESC

## 3.4 change vehicle_count (optional)

In [ ]:
-- Step 1: Add a new column for float values
ALTER TABLE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
ADD COLUMN vehicle_count_float FLOAT64;

-- Step 2: Populate the new column with float values from vehicle_count
UPDATE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
SET vehicle_count_float = CAST(vehicle_count AS FLOAT64)
WHERE vehicle_count IS NOT NULL; -- mandatory `where`

-- Step 3: Update the specific value from 3 to 3.5 in the new float column
UPDATE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
SET vehicle_count_float = 3.5
WHERE vehicle_count = 3;

In [ ]:
-- Step 3 only for future updates  

## 3.5 VA

In [ ]:
-- Step 1: Add a new column for vehicle availability
ALTER TABLE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
ADD COLUMN va_ac3 STRING; -- VA for HH with adult_count <= 3

-- Step 2: Populate the new column based on vehicle_count and adult_count
UPDATE `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
SET va_ac3 = CASE
  WHEN vehicle_count = 0 THEN 'zv'
  WHEN vehicle_count > 0 AND vehicle_count < adult_count THEN 'iv'
  WHEN vehicle_count >= adult_count THEN 'sv'
  ELSE NULL  -- This covers any cases that don't match the above conditions
END
WHERE adult_count <= 3;


> Q: WHEN adult_count > 3 AND vehicle_count = 0, do we get 'zv'?

    No.

## 3.6 VA groups

In [ ]:
SELECT
  mpo,
  va_ac3,
  COUNT(household_id) AS num_households
FROM
  `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
GROUP BY
  mpo, va_ac3
ORDER BY
  mpo, va_ac3;


> output: 'data\va_ac3_mpo_rep23_v0816.csv'

note

In [ ]:
-- check NULL
SELECT
  household_id, adult_count, vehicle_count
FROM
  `project-using-replica.tdm_calib23.adultCount_mpo_23Q4`
WHERE 
  va_ac3 IS NULL AND adult_count < 4
LIMIT 100;

-- There is no data to display.


> `va_ac3 IS NULL` means `adult_count >= 4`